In [1]:
from pathlib import Path

import pandas as pd

In [2]:
def merge_epochs(df: pd.DataFrame):
    merged = df.groupby("epoch").agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else None).reset_index()

    # Save the cleaned version
    return merged

In [3]:
def load_metrics_from_folder(folder_path: Path) -> dict:
    merged_results = {}

    # Look for all CSV files in the folder
    for file in folder_path.glob("*.csv"):
        model_name = file.stem

        # Read and merge metrics
        df = pd.read_csv(file)
        merged = merge_epochs(df)

        # Add to dictionary
        merged_results[model_name] = merged

    return merged_results

# Example usage
folder = Path("./data/results")

results_dict = load_metrics_from_folder(folder)

In [10]:
keys_to_process = [
    'rings_color', 'rings_color_new', 
    'rings_gray', 'rings_gray_new',
    'cocahis_color', 'cocahis_color_new',
    'cocahis_gray', 'cocahis_gray_new'
]

# Initialize an empty DataFrame to collect the results
best_results_df = pd.DataFrame()

for key in keys_to_process:
    df = results_dict[key]
    
    best_dice_epoch = df['val_dice'].idxmax()
    best_loss_epoch = df['val_loss'].idxmin()
    
    best_dice_row = df.iloc[best_dice_epoch].copy()
    best_dice_row.name = f'{key}_best_dice'
    
    best_loss_row = df.iloc[best_loss_epoch].copy()
    best_loss_row.name = f'{key}_best_loss'
    
    best_results_df = pd.concat([best_results_df, best_dice_row.to_frame().T, best_loss_row.to_frame().T])

# Show final collected DataFrame
display(best_results_df)


,epoch,step,train_epoch_time,train_loss,val_dice,val_epoch_time,val_iou,val_loss
rings_color_best_dice,47.0,1391.0,108.212143,0.447639,0.608481,16.933426,0.599883,0.882945
rings_color_best_loss,83.0,2435.0,108.301125,0.420048,0.498822,16.844345,0.462251,0.495588
rings_color_new_best_dice,42.0,2880.0,46.430832,0.448818,0.591322,7.673621,0.544074,0.527975
rings_color_new_best_loss,90.0,6096.0,46.478874,0.423550,0.458994,7.850736,0.422826,0.494639
rings_gray_best_dice,74.0,2174.0,33.193863,0.463388,0.336916,5.073780,0.289278,0.534097
rings_gray_best_loss,86.0,2522.0,33.331627,0.463646,0.323436,5.052083,0.277708,0.529013
rings_gray_new_best_dice,18.0,1272.0,29.348747,0.532648,0.523636,3.078266,0.521914,0.864595
rings_gray_new_best_loss,93.0,6297.0,29.151155,0.465790,0.348708,3.014453,0.304789,0.524692
cocahis_color_best_dice,9.0,19.0,13.595434,0.505520,0.567617,2.250940,0.427073,0.499085
cocahis_color_best_loss,18.0,37.0,13.834900,0.468532,0.500260,2.244825,0.368412,0.489381


In [12]:
best_results_df.to_csv("comparison.csv", index=True)